# Binary Files

Last lesson, we saw how computers use bits to represent information.  In this lesson, we will explore how to record those bits into permanent storage.

## When is a '1' not a 1?

Try this:

In [ ]:
%%file l03_ex01.c

#include <stdio.h>

int main()
{
    printf( "%d\n", ('1'==1) );
    return 0;
}

In [ ]:
%%bash
gcc -ansi -Wall -pedantic l03_ex01.c -o l03_ex01

In [ ]:
%%bash
./l03_ex01

0 means "False", so '1' isn't the same a 1?

Write a program that prints out the numerical value of '1'.

In [ ]:
%%file l03_ex02.c
#include <stdio.h>

/* add code here */


In [ ]:
%%bash
gcc -ansi -Wall -pedantic l03_ex02.c -o l03_ex02

In [ ]:
%%bash
./l03_ex02

Look at the following program which prints the square root of 2.

In [ ]:
%%file l03_ex03.c

#include <stdio.h>
#include <math.h>

/* add code here */

int main()
{
    double root2;
    
    root2 = sqrt( 2.0 );
    printf( "%100.98f", root2 );
    
    return 0;
}

## Important:  Link the math library with -lm

In [ ]:
%%bash
gcc -ansi -Wall -pedantic l03_ex03.c -lm -o l03_ex03

In [ ]:
%%bash
./l03_ex03


It shows 53 digits of precision.  To print it out takes 54 characters (including the decimal point).

Look at the following code.

In [ ]:
%%file l03_ex04.c

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <string.h>

int main( int argc, char **argv )
{
    int bitno, bit;
    double root2;
    unsigned long data;
    
    printf( "sizeof(double)=%ld, sizeof(unsigned long)=%ld\n", sizeof(double), sizeof(unsigned long) );
    
    root2 = sqrt( 2.0 );
    memcpy( &data, &root2, sizeof(double) );  /* copy memory from root2 to data so we can inspect it */
    
    printf( "%54.52f = %ld\n", root2, data );

    for (bitno=sizeof(double)*8-1;bitno>=0;bitno--)
    {
        bit = (data >> bitno)&1;
        printf( "%d", bit );
    }
    printf( "\n" );
    return 0;
}

In [ ]:
%%bash
gcc -ansi -Wall -pedantic l03_ex04.c -lm -o l03_ex04

In [ ]:
%%bash
./l03_ex04

It takes 64 bits, or 8 bytes, to encode the double precision floating point number.  So it can be stored using only 8 bytes, vs the 54 characters that would be used to print out all of the significant digits.

But storage isn't the only reason for using the 8 byte representation.  The CPU chips in the computer can do 8 byte operations natively.  That means they can do multiplication in on quick operation if the data is in the computer's native format.  If you used the text representation, you would have to first convert the multiplier and the multiplicand from text to native format, then multiply and then convert back.

The printf and scanf functions in C, make this every easy to do from the programmer's perspective, but it takes extra work from the computer.

When we save informtion from our programs, we can write that information to a file.  We have a choice to write out that information as text, or in the computer's native format.  Neither option is correct, neither is incorrect.

Some reasons to write your file in binary (native) format include:
<ul>
    <li> you want to read and write the file quickly, </li>
    <li> you want to minimize file size, </li>
    <li> your file consists of many records of identical format and size, </li>
    <li> you want to be able to jump to one particular location in your file, or </li>
    <li> you don't want other people to be able to easily read your file format. </li>
</ul>
Some reasons to write your file in text include:
<ul>
    <li> you want humans to be able to read the contents of your file without having to use your program, </li>
    <li> you want programmers of other programs to be able to easily understand and use your file, or </li>
    <li> your file contains variable lenth records (esp. text). </li>
</ul>

Ok, let's do an example.  Consider the following record structure:

In [ ]:
%%file record.h

#include <stdio.h>

/* example header file */

struct record
{
    int PLU;
    char comodity[64];
    char variety[64];
    char size[32];
    double price;
};

void read_text( FILE *fp, struct record *r );
void print_record( struct record r );

This is a record structure to store information about grocery store fruits and vegetables.  PLU is is the product look-up code for the grocery item.  "comodity" is the written name of the item. And, price is the price per kg.  Here is a text file containing some information:

In [ ]:
%%file plu.txt
4011,BANANAS,Yellow,All,0.12
3071,APPLES,Granny Smith,All,0.43
4111,APPLES,Crispin/Mutsu,Large,0.33
3283,APPLES,Honeycrisp,Large,0.32
4154,APPLES,McIntosh,Large,0.79
4048,LIMES,Regular,All,0.43
4307,LONGAN,,All,0.99
4256,CARAMBOLA,,ALL,0.87
4645,MUSHROOMS,Button,Small,0.22

Let's write some code to read that file.

In [ ]:
%%file record.c

#include "record.h"

void read_text( FILE *fp, struct record *r )
{
    char line[81];
    
    fgets( line, 81, fp );
    sscanf( line, "%d,%64[^,],%64[^,],%32[^,],%lf", 
               &(r->PLU), (r->comodity), (r->variety), (r->size), &(r->price) );
    
    
}

void print_record( struct record r )
{
    printf( "PLU:      %d\n", r.PLU );
    printf( "comodity: %s\n", r.comodity );
    printf( "variety:  %s\n", r.variety );
    printf( "size:     %s\n", r.size );
    printf( "price:    %.2f\n\n", r.price );
}

In [ ]:
%%file l03p01.c
#include <stdio.h>
#include <stdlib.h>
#include "record.h"

/* file that reads txt file and prints it to the screen */

int main( int argc, char **argv )
{
    FILE *fp;

    struct record r;
    
    if (argc!=2)
    {
        fprintf( stderr, "Usage: %s <filename>\n", argv[0] );
        exit(-1);
    }
    fp = fopen( argv[1], "r" );
    while ( !feof( fp ) )
    {
        
        read_text( fp, &r );
        print_record( r );
        
        
    }
    fclose( fp );
    
    return 0;
}

In [ ]:
%%bash
gcc -ansi -Wall -pedantic -c record.c -o record.o
gcc -ansi -Wall -pedantic l03p01.c record.o -o l03p01

In [ ]:
%%bash
./l03p01 plu.txt

Now we will create the following programs:

<ol>
    <li> A program which reads the text file and writes a binary file. </li>
    <li> A program which reads a binary file and prints the contents. </li>
    <li> A program which reads a binary file and writes a text file. </li>
</ol>

Next we will write and read the file in binary.

The commands we need for this are:

fwrite and fread

In [ ]:
%%bash 
man fwrite

In [ ]:
%%bash
man fread